In [4]:
import sys
import os
sys.path.append("/content/ml-project-2-pytyeee/")
REPO_DIR = "ml-project-2-pytyeee/"

In [2]:
from scripts.preprocessing import *
from helpers.model_handler import *

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [6]:
test_dir = REPO_DIR + "dataset/test_set_images/"
test_images_filename = test_dir

TESTING_SIZE = 50

test_data = extract_data(test_images_filename, TESTING_SIZE, test=True)
test_data = torch.from_numpy(np.transpose(test_data, (0, 3, 1, 2)))
print(f"Test shape: {test_data.shape}")

test_data = test_data.to(DEVICE)

Loading ../dataset/test_set_images/test_1/test_1.png
Loading ../dataset/test_set_images/test_50/test_50.png
Test shape: torch.Size([50, 3, 608, 608])


In [7]:
model = load_model(REPO_DIR + 'model_saves/model_2023-12-07_0.pt' )

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
predictions = model(test_data)
pred = torch.where(predictions > 0.5, 1., 0.)